In [1]:
# Importation des packages python
import numpy as np
import pandas as pd
import json # Pour pouvoir faire les chargements de fichiers au format json

In [2]:
# Charger le fichier JSON des quartiers contenant les délimitations géographiques des quartiers ...
# ... sous forme de suite de points longitude, latitude formant une structure Polygone qui permettra la recherche d'une adresse sous forme de point géographique
with open('quartiers-de-democratie-locale.json', "r") as fichier_quartiers:
    data_quartiers = json.load(fichier_quartiers)

In [3]:
# Charger le fichier JSON des nuémeros de rues contenant les points géographiques de chaque adresse ...
# ... sous la forme d'un point longitude, latitude qui permettra la recherche d'une adresse dans les Polygones des quartiers
with open('numero-de-rue.json', "r") as fichier_rues:
    data_rues = json.load(fichier_rues)

FORMATAGE DES DONNEES AVANT TRAITEMENT POUR METTRE SOUS FORME DE TABLEAU (dataframe) LES DONNEES CHARGEES (json)

AVANT : type(data_xxx) : list / type(data_xxx[0]) : dict

APRES : dataframe
- puis suppression dans df_quartiers de [[ ]] en trop et remplacement de [long,lat] par (long,lat),
- puis suppression dans df_rues de [] en trop,
- puis suppression des adresses dans df_rues dont commune <> TOULOUSE car adresses non concernées par recherche quartier

In [4]:
# La fonction pd.json_normalize(data) normalise les données JSON semi-structurées en table plate dataframe.
df_quartiers = pd.json_normalize(data_quartiers)
df_rues = pd.json_normalize(data_rues)

In [6]:
# Affichage de contrôle de df_quartiers
print(df_quartiers.to_string())

    quartier                                                                    nom_quartier      maire_de_quartier  geo_point_2d.lon  geo_point_2d.lat geo_shape.type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [7]:
# Affichage de contrôle df_rues ... attention à limiter l'affichage avec head() car près de 70000 lignes !
print(df_rues.head(10).to_string())

   numero repetition num_long                 libelle           sti       rivoli  code_postal        commune  code_insee  geo_point_2d.lon  geo_point_2d.lat geo_shape.type             geo_shape.geometry.coordinates geo_shape.geometry.type
0       6       None     0006   RUE DE L AMOUR TROMPE  311570000228  3101571020A        31270        CUGNAUX       31157          1.350478         43.556520        Feature  [[1.350477977713465, 43.556519564469035]]              MultiPoint
1      47       None     0047       AVENUE D OUESSANT  311490000335  3101490157X        31770      COLOMIERS       31149          1.323259         43.615302        Feature   [[1.323259161106773, 43.61530179303467]]              MultiPoint
2      11       None     0011    RUE ARISTIDE MAILLOL  310690000240  3100690044X        31700        BLAGNAC       31069          1.376447         43.644155        Feature   [[1.376447240004688, 43.64415531820807]]              MultiPoint
3      17       None     0017           CHEM

In [8]:
# Suppression des [[ ]] et remplacement des [,] par (,) dans df_quartiers
from itertools import chain

def transformer_geozone(df_quartiers):
  # Suppression dans df_quartiers des [[ et ]] en trop dans df["geo_shape.geometry.coordinates"]
  # [list(chain(*x)) for x in df_quartiers["geo_shape.geometry.coordinates"]] ...
  # ... Cette expression est une boucle qui parcourt chaque élément de la colonne et effectue les transformations suivantes :
  #     *x : Décompresse chaque élément de la liste (qui est une liste de listes) en une seule liste.
  #     list(chain(*x)) : Fusionne les éléments décompressés en une seule liste.
  #     =: Assigne le résultat de l'expression (une liste de listes aplaties) à la colonne geo_shape.geometry.coordinates.
  df_quartiers["geo_shape.geometry.coordinates"] = [list(chain(*x)) for x in df_quartiers["geo_shape.geometry.coordinates"]]
  df_quartiers["geo_shape.geometry.coordinates"] = [list(chain(*x)) for x in df_quartiers["geo_shape.geometry.coordinates"]]
  # Remplacement des [x,y] par (x,y) dans df_quartiers["geo_shape.geometry.coordinates"] pour pouvoir avoir le format POLYGON attendu pour la recherche d'un point géographique dans le POLYGON
  # df_quartiers["geo_shape.geometry.coordinates"] = df_quartiers["geo_shape.geometry.coordinates"].apply(lambda x: [(y, z) for y, z in x])
  df_quartiers['geo_shape.geometry.coordinates'] = df_quartiers['geo_shape.geometry.coordinates'].replace('], [', '), (')
  return df_quartiers

df_quartiers = transformer_geozone(df_quartiers.copy())

# Affichage de contrôle
print(df_quartiers.to_string())

    quartier                                                                    nom_quartier      maire_de_quartier  geo_point_2d.lon  geo_point_2d.lat geo_shape.type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [9]:
# Suppression des [] en trop dans df_rues pour besoin de formatage avant recherche
def transformer_geozone(df_rues):
  df_rues["geo_shape.geometry.coordinates"] = [list(chain(*x)) for x in df_rues["geo_shape.geometry.coordinates"]]
  return df_rues

df_rues = transformer_geozone(df_rues.copy())

# Affichage de contrôle
print(df_rues.head(10).to_string())

   numero repetition num_long                 libelle           sti       rivoli  code_postal        commune  code_insee  geo_point_2d.lon  geo_point_2d.lat geo_shape.type           geo_shape.geometry.coordinates geo_shape.geometry.type
0       6       None     0006   RUE DE L AMOUR TROMPE  311570000228  3101571020A        31270        CUGNAUX       31157          1.350478         43.556520        Feature  [1.350477977713465, 43.556519564469035]              MultiPoint
1      47       None     0047       AVENUE D OUESSANT  311490000335  3101490157X        31770      COLOMIERS       31149          1.323259         43.615302        Feature   [1.323259161106773, 43.61530179303467]              MultiPoint
2      11       None     0011    RUE ARISTIDE MAILLOL  310690000240  3100690044X        31700        BLAGNAC       31069          1.376447         43.644155        Feature   [1.376447240004688, 43.64415531820807]              MultiPoint
3      17       None     0017           CHEMIN REBOU

In [10]:
# Suppression des adresses hors commune Toulouse car non concernées par la recherche du quartier
df_rues = df_rues[df_rues['commune'] == 'TOULOUSE']
print(df_rues.head(10).to_string())

    numero repetition num_long                  libelle           sti       rivoli  code_postal   commune  code_insee  geo_point_2d.lon  geo_point_2d.lat geo_shape.type           geo_shape.geometry.coordinates geo_shape.geometry.type
3       17       None     0017            CHEMIN REBOUL  315555779227  3105557336K        31100  TOULOUSE       31555          1.376699         43.581693        Feature   [1.376698810722346, 43.58169278322894]              MultiPoint
9        3       None     0003   RUE GEORGES COURTELINE  315558391851  3105553850W        31100  TOULOUSE       31555          1.415388         43.591612        Feature   [1.415387910507514, 43.59161180871026]              MultiPoint
11      14       None     0014  CHEMINEMENT HENRI HERTZ  315558371732  3105554223B        31100  TOULOUSE       31555          1.398589         43.579427        Feature   [1.398589181937803, 43.57942675914894]              MultiPoint
14      27       None     0027             RUE DU CONGO  3155519

In [11]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
## création des variables Point et Polygon pour pouvoir faire la recherche avec les méthodes <within> sur les variables de type Point et Polygon

# import bibliothèque pour les fonctions de recherche géographiques
from shapely.geometry import Polygon, Point

# Créez des objets Point pour les coordonnées des rues
df_rues['point_rue'] = df_rues['geo_shape.geometry.coordinates'].apply(Point)

# Créez des objets Polygon pour les coordonnées des quartiers
df_quartiers['polygone'] = df_quartiers['geo_shape.geometry.coordinates'].apply(Polygon)

PARTIE TRAITEMENT POUR AJOUT DE LA COLONNE QUARTIER DANS LE DATAFRAME df_rues POUR TOUTES LES ADRESSES DE TOULOUSE

In [13]:
# Fonction pour vérifier si un point est dans un polygone
def est_dans_polygones(point, polygone):
    return point.within(polygone) # or point.touches(polygon) si on veut aussi inclure la bordure du POLYGON mais à priori pas besoin pour notre cas

# Appliquez la fonction pour chaque rue et quartier
df_rues['quartier'] = df_rues.apply(lambda row_rue: next((row_q['quartier'] for _, row_q in df_quartiers.iterrows() if est_dans_polygones(row_rue['point_rue'], row_q['polygone'])), None), axis=1)

# Supprimez la colonne temporaire 'point_rue'
df_rues.drop(columns=['point_rue'], inplace=True)

# Contrôle
print(df_rues.head(20).to_string())

    numero repetition num_long                            libelle           sti       rivoli  code_postal   commune  code_insee  geo_point_2d.lon  geo_point_2d.lat geo_shape.type           geo_shape.geometry.coordinates geo_shape.geometry.type  quartier
3       17       None     0017                      CHEMIN REBOUL  315555779227  3105557336K        31100  TOULOUSE       31555          1.376699         43.581693        Feature   [1.376698810722346, 43.58169278322894]              MultiPoint        18
9        3       None     0003             RUE GEORGES COURTELINE  315558391851  3105553850W        31100  TOULOUSE       31555          1.415388         43.591612        Feature   [1.415387910507514, 43.59161180871026]              MultiPoint        16
11      14       None     0014            CHEMINEMENT HENRI HERTZ  315558371732  3105554223B        31100  TOULOUSE       31555          1.398589         43.579427        Feature   [1.398589181937803, 43.57942675914894]              Multi

PREPARATION DONNEES POUR CREATION DU FICHIER RESULTAT CONTENANT RESULTATS + AJOUT COLONNE adresse POUR AVOIR ADRESSE COMPLETE POUR FAIRE DE LA VISUALISATION SUR POWER BI :-)


In [14]:
# On ne garde que les champs qui nous intéressent
df_rues_quartiers = df_rues.filter(items=['numero', 'repetition', 'libelle', 'code_postal', 'commune', 'quartier'])

df_rues_quartiers['adresse'] = df_rues_quartiers['numero'].astype(str)
for index in df_rues_quartiers.index:
  if df_rues_quartiers.loc[index, 'repetition'] != None:
    df_rues_quartiers.loc[index, 'adresse'] += df_rues_quartiers.loc[index, 'repetition']
df_rues_quartiers['adresse'] += ' ' + df_rues_quartiers['libelle'] + ' ' + df_rues_quartiers['code_postal'].astype(str) + ' ' + df_rues_quartiers['commune']

display(df_rues_quartiers.head(10))

,numero,repetition,libelle,code_postal,commune,quartier,adresse
3,17,None,CHEMIN REBOUL,31100,TOULOUSE,18,17 CHEMIN REBOUL 31100 TOULOUSE
9,3,None,RUE GEORGES COURTELINE,31100,TOULOUSE,16,3 RUE GEORGES COURTELINE 31100 TOULOUSE
11,14,None,CHEMINEMENT HENRI HERTZ,31100,TOULOUSE,17,14 CHEMINEMENT HENRI HERTZ 31100 TOULOUSE
14,27,None,RUE DU CONGO,31500,TOULOUSE,11,27 RUE DU CONGO 31500 TOULOUSE
16,77,None,RUE LOUIS PLANA,31500,TOULOUSE,10,77 RUE LOUIS PLANA 31500 TOULOUSE
24,11,None,RUE DES CHAMOIS,31200,TOULOUSE,9,11 RUE DES CHAMOIS 31200 TOULOUSE
26,46,None,RUE ROBERT MESURET,31100,TOULOUSE,17,46 RUE ROBERT MESURET 31100 TOULOUSE
29,32,None,RUE LOUVOIS,31500,TOULOUSE,10,32 RUE LOUVOIS 31500 TOULOUSE
31,42,None,RUE MIREILLE SORGUE,31100,TOULOUSE,14,42 RUE MIREILLE SORGUE 31100 TOULOUSE
33,34,None,RUE VICENTE LOPEZ TOVAR,31100,TOULOUSE,14,34 RUE VICENTE LOPEZ TOVAR 31100 TOULOUSE


In [15]:
# Harmonistaion des abbréviations dans fichier fourni car il existe des adresses avec abbréviation et sans (exemple ST ou SAINT coexiste dans le même fichier!) 
df_rues_quartiers["adresse"] = df_rues_quartiers["adresse"].str.replace(" ST ", " SAINT ")
df_rues_quartiers["adresse"] = df_rues_quartiers["adresse"].str.replace(" STE ", " SAINTE ")
df_rues_quartiers["adresse"] = df_rues_quartiers["adresse"].str.replace(" PR ", " PROFESSEUR ")
df_rues_quartiers["adresse"] = df_rues_quartiers["adresse"].str.replace(" DR ", " DOCTEUR ")
df_rues_quartiers["adresse"] = df_rues_quartiers["adresse"].str.replace(" GEN ", " GENERAL ")
df_rues_quartiers["adresse"] = df_rues_quartiers["adresse"].str.replace(" FITZG ", " FITZGERALD ") # coquille du fichier fourni :-)
display(df_rues_quartiers.head(10))

,numero,repetition,libelle,code_postal,commune,quartier,adresse
3,17,None,CHEMIN REBOUL,31100,TOULOUSE,18,17 CHEMIN REBOUL 31100 TOULOUSE
9,3,None,RUE GEORGES COURTELINE,31100,TOULOUSE,16,3 RUE GEORGES COURTELINE 31100 TOULOUSE
11,14,None,CHEMINEMENT HENRI HERTZ,31100,TOULOUSE,17,14 CHEMINEMENT HENRI HERTZ 31100 TOULOUSE
14,27,None,RUE DU CONGO,31500,TOULOUSE,11,27 RUE DU CONGO 31500 TOULOUSE
16,77,None,RUE LOUIS PLANA,31500,TOULOUSE,10,77 RUE LOUIS PLANA 31500 TOULOUSE
24,11,None,RUE DES CHAMOIS,31200,TOULOUSE,9,11 RUE DES CHAMOIS 31200 TOULOUSE
26,46,None,RUE ROBERT MESURET,31100,TOULOUSE,17,46 RUE ROBERT MESURET 31100 TOULOUSE
29,32,None,RUE LOUVOIS,31500,TOULOUSE,10,32 RUE LOUVOIS 31500 TOULOUSE
31,42,None,RUE MIREILLE SORGUE,31100,TOULOUSE,14,42 RUE MIREILLE SORGUE 31100 TOULOUSE
33,34,None,RUE VICENTE LOPEZ TOVAR,31100,TOULOUSE,14,34 RUE VICENTE LOPEZ TOVAR 31100 TOULOUSE


In [16]:
# Création du fichier csv pour usage externe 
df_rues_quartiers.to_csv('numero-de-rue-quartier.csv')